In [1]:
import numpy as np # For numerical analysis and work
import pandas as pd # For summarizing the code and classes

In [6]:
class BaseLayer:
    def __init__(self):
        self.inputs = None
        self.output = None
        
        self.d_inputs = None

    def forward(self, inputs):
        ...

    def backward(self, d_values):
        ...

    def __call__(self, inputs, backward_pass=False):
        if backward_pass:
            self.backward(inputs)
        else:
            self.forward(inputs)
        
class Dense(BaseLayer):
    def __init__(self, d_out, d_in=None, keepbias=True, activation='linear'):
        self.d_out = d_out
        self.d_in = d_in
        
        self.keepbias = keepbias
        self.activation = activation
        
        if self.d_in:
            self.__init_params_(self.d_in)

    def __init_params_(self, d_in):
        self.d_in = d_in
        self.weights = np.random.randn(self.d_in, self.d_out) / np.sqrt(d_in)
        if self.keepbias:
            self.biases = np.random.randn(self.d_out)

    @property
    def parameters(self):
        return [self.weights, self.biases] if self.keepbias else [self.weights]

    @parameters.setter
    def parameters(self, value):
        self.weights = value[0]
        if self.keepbias:
            self.biases = value[1]

    @property
    def d_parameters(self):
        return [self.d_weights, self.d_biases] if self.keepbias else [self.d_weights]

    @d_parameters.setter
    def d_parameters(self, value):
        self.d_weights = value[0]
        if self.keepbias:
            self.d_biases = value[1]
        

    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs @ self.weights + self.biases

    def backward(self, d_values):
        n = len(d_values.shape)
        self.d_weights = self.inputs.transpose((*range(n-2), n-1, n-2)) @ d_values
        if self.keepbias:
            self.d_biases = np.sum(self.inputs, axis=range(n-1))

        self.d_inputs = d_values @ self.weights.T

    def summary(self):
        {
            "name": "Dense",
            "n_params": self._n_params,
            "activation": self.activation,
            "keepbias": self.keepbias,
        }

    @property
    def _n_params(self):
        return self.d_out*self.d_in+self.d_out if self.keepbias else self.d_out*self.d_in

In [9]:
class ReLU(BaseLayer):
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(inputs, 0)

    def backward(self, d_values):
        self.d_inputs = np.where(self.inputs>0, 1, 0)*d_values

class Linear(BaseLayer):
    def forward(self, inputs):
        self.inputs = inputs
        self.output = self.inputs

    def backward(self, d_values):
        self.d_inputs = d_values

class LeakyReLU(BaseLayer):
    def __init__(self, leaky=0.001):
        super().__init__()
        self.leaky_constant = leaky
    
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(inputs, inputs*self.leaky_constant)

    def backward(self, d_values):
        self.d_inputs = np.where(self.inputs>0, 1, self.leaky_constant)*d_values

class Sigmoid(BaseLayer):
    epsilon = 1e-8
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1/(1+np.exp(-inputs)+self.epsilon)

    def backward(self, d_values):
        self.d_inputs = (self.output*(1-self.output))*d_values
        
class TanH(BaseLayer):
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, d_values):
        self.d_inputs = (1-self.output*self.output)*d_values

class SoftMax(BaseLayer):
    def __init__(self, loss_c=True):
        super().__init__()
        self.loss_c = loss_c

    def forward(self, inputs):
        self.inputs = inputs
        n = len(inputs.shape)
        max_vals = np.max(inputs, axis=n-1, keepdims=True)
        exp_vals = np.exp(inputs-max_vals)
        self.output = np.sum(exp_vals, axis=n-1, keepdims=True)

    def backward(self, d_values):
        if not self.loss_c:
            output_shape = d_values.shape
            d_out = output_shape[-1]
            M = np.tile(self.output, d_out).reshape(*output_shape, d_out)
            I = np.eye(d_out)
            Z = M * (I - M.T)
    
            self.output = np.sum(np.multiply(Z, d_values[..., np.newaxis, :]), axis=-1)
        else:
            self.output = d_values

In [11]:
def activation_parser(activation, *args, **kwargs):
    match activation:
        case "relu": activator = ReLU
        case "linear": activator = Linear
        case "tanh": activator = TanH
        case "sigmoid": activator = Sigmoid
        case "leaky-relu": activator = LeakyReLU
        case "softmax": activator = SoftMax

    return activator(*args, **kwargs)

In [ ]:
class LSTM:
    def __init__(self, d_in, hidden_state, call_state, d_out):
        self.dn1 = Dense(d_out=call_state, d_in=d_in)
        self.dh1 = Dense(d_out=call_state, d_in=hidden_state)

        self.dn2 = Dense(d_out=call_state, d_in=d_in)
        self.dh2 = Dense(d_out=call_state, d_in=hidden_state)
        
        self.dn1 = Dense(d_out=call_state, d_in=d_in)
        self.dh1 = Dense(d_out=call_state, d_in=hidden_state)

        self.dn1 = Dense(d_out=call_state, d_in=d_in)
        self.dh1 = Dense(d_out=call_state, d_in=hidden_state)